## Testing the z-Entropy PML prior-estimate on the Feder testcase
for different samplings of simulated data and z-samplings in the entropy monte carlo approximation.

In [ ]:
using GynC
using Plots

In [74]:
for zmult = [1,5,10,30,50,100,200, 300, 300, 300]
  rho_std = 5
  nx = 300
  #zmult = 300
  ndata = 200

  xs, ys, datas, zs = GynC.Federn.federexperiment(nx=nx, zmult=zmult, ndata=ndata, rho_std=rho_std, xmin=1, xmax=110);
  w0 = GynC.Federn.wbeta(xs, 110+0.1); # add 0.1 to circumvent 0 weight;

  dhz = w->GynC.Dhz(w, ys, zs, rho_std)
  dll = w->GynC.dlogl(w, datas, ys, rho_std)

  reg = 0.5
  niter = 50
  h = 0.0001

  @time ws = GynC.gradientascent(w->dhz(w) * reg + (1-reg)*dll(w), w0, niter, h, autodiff=false)
  plot(xs, ws, yaxis="\pi", title="$zmult zsamples") |> display

end

INFO: computing likelihood matrix
INFO: computing likelihood matrix


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 0.020 
 
 
 1 zsamples 
 
 
 pi 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 67.5962,371.572 69.3637,368.258 71.1311,365.012 72.8986,361.833 74.666,358.72 76.4335,355.673 78.2009,352.691 79.9684,349.774 81.7358,346.919 83.5033,344.128 
 85.2707,341.399 87.0382,338.732 88.8056,336.126 90.5731,333.58 92.3405,331.094 94.1079,328.667 95.8754,326.298 97.6428,323.987 99.4103,321.733 101.178,319.535 
 102.945,317.394 104.713,315.307 106.48,313.275 108.248,311.297 110.015,309.373 111.782,307.501 113.55,305.681 115.317,303.913 117.085,302.195 118.852,300.528 
 120.62,298.911 122.387,297.343 124.155,295.823 125.922,294.351 127.689,292.926 129.457,291.548 131.224,290.216 132.992,288.93 134.759,287.689 136.527,286.492 
 138.294,285.339 140.062,284.229 141.829,283.162 143.596,282.137 145.364,281.153 147.131,280.211 148.899,279.309 150.666,278.447 152.434,277.624 154.201,276.841 
 155.969,276.095 157.736,275.387 159.503,274.717 161.271,274.083 163.038,273.485 164.806,272.923 166.573,272.396 168.341,271.904 170.108,271.445 171.876,271.021 
 173.643,270.629 175.411,270.27 177.178,269.943 178.945,269.647 180.713,269.383 182.48,269.149 184.248,268.945 186.015,268.771 187.783,268.625 189.55,268.509 
 191.318,268.42 193.085,268.359 194.852,268.326 196.62,268.319 198.387,268.338 200.155,268.383 201.922,268.454 203.69,268.549 205.457,268.669 207.225,268.812 
 208.992,268.98 210.759,269.17 212.527,269.383 214.294,269.618 216.062,269.875 217.829,270.153 219.597,270.452 221.364,270.771 223.132,271.111 224.899,271.47 
 226.666,271.849 228.434,272.246 230.201,272.661 231.969,273.095 233.736,273.546 235.504,274.014 237.271,274.499 239.039,275.001 240.806,275.518 242.574,276.051 
 244.341,276.599 246.108,277.162 247.876,277.74 249.643,278.331 251.411,278.936 253.178,279.555 254.946,280.186 256.713,280.83 258.481,281.487 260.248,282.155 
 262.015,282.835 263.783,283.525 265.55,284.227 267.318,284.939 269.085,285.661 270.853,286.394 272.62,287.135 274.388,287.886 276.155,288.645 277.922,289.413 
 279.69,290.189 281.457,290.973 283.225,291.764 284.992,292.562 286.76,293.368 288.527,294.179 290.295,294.998 292.062,295.822 293.829,296.651 295.597,297.486 
 297.364,298.326 299.132,299.171 300.899,300.021 302.667,300.874 304.434,301.732 306.202,302.593 307.969,303.457 309.737,304.325 311.504,305.195 313.271,306.068 
 315.039,306.943 316.806,307.821 318.574,308.7 320.341,309.58 322.109,310.462 323.876,311.345 325.644,312.228 327.411,313.112 329.178,313.997 330.946,314.881 
 332.713,315.766 334.481,316.649 336.248,317.533 338.016,318.415 339.783,319.297 341.551,320.177 343.318,321.055 345.085,321.932 346.853,322.807 348.62,323.68 
 350.388,324.55 352.155,325.418 353.923,326.284 355.69,327.146 357.458,328.005 359.225,328.861 360.992,329.714 362.76,330.563 364.527,331.408 366.295,332.249 
 368.062,333.086 369.83,333.919 371.597,334.747 373.365,335.57 375.132,336.389 376.9,337.203 378.667,338.011 380.434,338.814 382.202,339.612 383.969,340.405 
 385.737,341.191 387.504,341.972 389.272,342.747 391.039,343.516 392.807,344.278 394.574,345.034 396.341,345.784 398.109,346.527 399.876,347.263 401.644,347.993 
 403.411,348.716 405.179,349.431 406.946,350.14 408.714,350.841 410.481,351.535 412.248,352.221 414.016,352.901 415.783,353.572 417.551,354.236 419.318,354.892 
 421.086,355.54 422.853,356.18 424.621,356.812 426.388,357.436 428.155,358.052 429.923,358.66 431.69,359.26 433.458,359.851 435.225,360.434 436.993,361.009 
 438.76,361.575 440.528,362.133 442.295,362.682 444.063,363.222 445.83,363.754 447.597,364.277 449.365,364.792 451.132,365.298 452.9,365.795 454.667,366.284 
 456.435,366.763 458.202,367.234 459.97,367.697 461.737,368.15 463.504,368.595 465.272,369.03 467.039,369.457 468.807,369.876 470.5

  0.358218 seconds (1.15 M allocations: 103.925 MB, 52.83% gc time)


INFO: computing likelihood matrix
INFO: computing likelihood matrix


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 5 zsamples 
 
 
 pi 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 67.5962,369.977 69.3637,366.104 71.1311,362.31 72.8986,358.594 74.666,354.955 76.4335,351.394 78.2009,347.908 79.9684,344.498 81.7358,341.161 83.5033,337.899 
 85.2707,334.709 87.0382,331.591 88.8056,328.545 90.5731,325.569 92.3405,322.663 94.1079,319.826 95.8754,317.057 97.6428,314.356 99.4103,311.721 101.178,309.153 
 102.945,306.649 104.713,304.21 106.48,301.835 108.248,299.523 110.015,297.274 111.782,295.086 113.55,292.959 115.317,290.892 117.085,288.884 118.852,286.936 
 120.62,285.045 122.387,283.212 124.155,281.435 125.922,279.715 127.689,278.05 129.457,276.439 131.224,274.882 132.992,273.379 134.759,271.928 136.527,270.529 
 138.294,269.181 140.062,267.884 141.829,266.636 143.596,265.438 145.364,264.289 147.131,263.187 148.899,262.133 150.666,261.125 152.434,260.164 154.201,259.248 
 155.969,258.376 157.736,257.549 159.503,256.765 161.271,256.024 163.038,255.325 164.806,254.668 166.573,254.053 168.341,253.477 170.108,252.941 171.876,252.445 
 173.643,251.987 175.411,251.567 177.178,251.185 178.945,250.839 180.713,250.53 182.48,250.257 184.248,250.019 186.015,249.815 187.783,249.645 189.55,249.509 
 191.318,249.405 193.085,249.334 194.852,249.295 196.62,249.287 198.387,249.309 200.155,249.362 201.922,249.444 203.69,249.556 205.457,249.696 207.225,249.864 
 208.992,250.059 210.759,250.282 212.527,250.53 214.294,250.805 216.062,251.105 217.829,251.43 219.597,251.78 221.364,252.154 223.132,252.55 224.899,252.97 
 226.666,253.413 228.434,253.877 230.201,254.362 231.969,254.869 233.736,255.397 235.504,255.944 237.271,256.511 239.039,257.097 240.806,257.702 242.574,258.325 
 244.341,258.965 246.108,259.623 247.876,260.298 249.643,260.99 251.411,261.697 253.178,262.42 254.946,263.158 256.713,263.911 258.481,264.678 260.248,265.459 
 262.015,266.254 263.783,267.061 265.55,267.882 267.318,268.714 269.085,269.558 270.853,270.414 272.62,271.281 274.388,272.158 276.155,273.046 277.922,273.943 
 279.69,274.85 281.457,275.766 283.225,276.691 284.992,277.624 286.76,278.566 288.527,279.515 290.295,280.471 292.062,281.434 293.829,282.404 295.597,283.38 
 297.364,284.362 299.132,285.35 300.899,286.342 302.667,287.34 304.434,288.342 306.202,289.349 307.969,290.359 309.737,291.373 311.504,292.391 313.271,293.411 
 315.039,294.434 316.806,295.459 318.574,296.487 320.341,297.516 322.109,298.547 323.876,299.579 325.644,300.612 327.411,301.645 329.178,302.679 330.946,303.713 
 332.713,304.746 334.481,305.779 336.248,306.812 338.016,307.843 339.783,308.874 341.551,309.902 343.318,310.929 345.085,311.954 346.853,312.977 348.62,313.997 
 350.388,315.015 352.155,316.029 353.923,317.041 355.69,318.049 357.458,319.053 359.225,320.054 360.992,321.05 362.76,322.043 364.527,323.03 366.295,324.014 
 368.062,324.992 369.83,325.965 371.597,326.933 373.365,327.896 375.132,328.852 376.9,329.804 378.667,330.749 380.434,331.688 382.202,332.62 383.969,333.546 
 385.737,334.466 387.504,335.378 389.272,336.284 391.039,337.183 392.807,338.074 394.574,338.958 396.341,339.834 398.109,340.703 399.876,341.563 401.644,342.416 
 403.411,343.261 405.179,344.097 406.946,344.926 408.714,345.745 410.481,346.556 412.248,347.359 414.016,348.153 415.783,348.937 417.551,349.713 419.318,350.48 
 421.086,351.238 422.853,351.986 424.621,352.725 426.388,353.454 428.155,354.175 429.923,354.885 431.69,355.586 433.458,356.277 435.225,356.959 436.993,357.63 
 438.76,358.292 440.528,358.944 442.295,359.586 444.063,360.217 445.83,360.839 447.597,361.451 449.365,362.052 451.132,362.644 452.9,363.225 454.667,363.796 
 456.435,364.357 458.202,364.907 459.97,365.447 461.737,365.977 463.504,366.497 465.272,367.006 467.039,367.506 468.807,367.994 470.574,368.473 47

...

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 0.020 
 
 
 300 zsamples 
 
 
 pi 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 67.5962,372.106 69.3637,368.98 71.1311,365.917 72.8986,362.918 74.666,359.981 76.4335,357.107 78.2009,354.293 79.9684,351.541 81.7358,348.848 83.5033,346.215 
 85.2707,343.64 87.0382,341.124 88.8056,338.665 90.5731,336.263 92.3405,333.917 94.1079,331.627 95.8754,329.393 97.6428,327.212 99.4103,325.086 101.178,323.013 
 102.945,320.992 104.713,319.024 106.48,317.107 108.248,315.241 110.015,313.425 111.782,311.659 113.55,309.942 115.317,308.274 117.085,306.653 118.852,305.081 
 120.62,303.555 122.387,302.075 124.155,300.641 125.922,299.252 127.689,297.908 129.457,296.608 131.224,295.352 132.992,294.138 134.759,292.967 136.527,291.838 
 138.294,290.75 140.062,289.703 141.829,288.696 143.596,287.729 145.364,286.801 147.131,285.912 148.899,285.061 150.666,284.248 152.434,283.472 154.201,282.733 
 155.969,282.029 157.736,281.362 159.503,280.729 161.271,280.131 163.038,279.567 164.806,279.037 166.573,278.54 168.341,278.075 170.108,277.643 171.876,277.242 
 173.643,276.872 175.411,276.534 177.178,276.225 178.945,275.946 180.713,275.697 182.48,275.476 184.248,275.284 186.015,275.119 187.783,274.982 189.55,274.872 
 191.318,274.789 193.085,274.731 194.852,274.7 196.62,274.693 198.387,274.711 200.155,274.754 201.922,274.82 203.69,274.91 205.457,275.023 207.225,275.159 
 208.992,275.316 210.759,275.496 212.527,275.697 214.294,275.919 216.062,276.161 217.829,276.423 219.597,276.705 221.364,277.007 223.132,277.327 224.899,277.666 
 226.666,278.023 228.434,278.398 230.201,278.79 231.969,279.199 233.736,279.624 235.504,280.066 237.271,280.524 239.039,280.997 240.806,281.485 242.574,281.988 
 244.341,282.505 246.108,283.036 247.876,283.581 249.643,284.139 251.411,284.71 253.178,285.293 254.946,285.889 256.713,286.497 258.481,287.116 260.248,287.746 
 262.015,288.388 263.783,289.039 265.55,289.701 267.318,290.373 269.085,291.055 270.853,291.745 272.62,292.445 274.388,293.153 276.155,293.869 277.922,294.594 
 279.69,295.326 281.457,296.065 283.225,296.812 284.992,297.565 286.76,298.325 288.527,299.091 290.295,299.863 292.062,300.64 293.829,301.423 295.597,302.211 
 297.364,303.003 299.132,303.8 300.899,304.602 302.667,305.407 304.434,306.216 306.202,307.028 307.969,307.844 309.737,308.662 311.504,309.484 313.271,310.307 
 315.039,311.133 316.806,311.96 318.574,312.79 320.341,313.62 322.109,314.452 323.876,315.285 325.644,316.119 327.411,316.953 329.178,317.787 330.946,318.622 
 332.713,319.456 334.481,320.29 336.248,321.123 338.016,321.956 339.783,322.787 341.551,323.618 343.318,324.447 345.085,325.274 346.853,326.099 348.62,326.923 
 350.388,327.744 352.155,328.563 353.923,329.379 355.69,330.193 357.458,331.004 359.225,331.811 360.992,332.616 362.76,333.416 364.527,334.214 366.295,335.007 
 368.062,335.797 369.83,336.582 371.597,337.364 373.365,338.141 375.132,338.913 376.9,339.681 378.667,340.443 380.434,341.201 382.202,341.954 383.969,342.702 
 385.737,343.444 387.504,344.18 389.272,344.911 391.039,345.637 392.807,346.356 394.574,347.069 396.341,347.777 398.109,348.478 399.876,349.172 401.644,349.861 
 403.411,350.542 405.179,351.218 406.946,351.886 408.714,352.548 410.481,353.202 412.248,353.85 414.016,354.491 415.783,355.124 417.551,355.75 419.318,356.369 
 421.086,356.981 422.853,357.585 424.621,358.181 426.388,358.77 428.155,359.351 429.923,359.925 431.69,360.49 433.458,361.048 435.225,361.598 436.993,362.14 
 438.76,362.674 440.528,363.201 442.295,363.719 444.063,364.229 445.83,364.73 447.597,365.224 449.365,365.71 451.132,366.187 452.9,366.656 454.667,367.117 
 456.435,367.569 458.202,368.014 459.97,368.45 461.737,368.878 463.504,369.297 465.272,369.708 467.039,370.111 468.807,370.506 470.57

 58.142351 seconds (189.51 M allocations: 17.745 GB, 34.63% gc time)


INFO: computing likelihood matrix
INFO: computing likelihood matrix


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 300 zsamples 
 
 
 pi 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 67.5962,367.4 69.3637,362.623 71.1311,357.943 72.8986,353.36 74.666,348.872 76.4335,344.479 78.2009,340.18 79.9684,335.973 81.7358,331.858 83.5033,327.834 
 85.2707,323.899 87.0382,320.054 88.8056,316.296 90.5731,312.626 92.3405,309.041 94.1079,305.542 95.8754,302.127 97.6428,298.795 99.4103,295.545 101.178,292.377 
 102.945,289.289 104.713,286.281 106.48,283.352 108.248,280.5 110.015,277.725 111.782,275.026 113.55,272.403 115.317,269.853 117.085,267.377 118.852,264.974 
 120.62,262.642 122.387,260.381 124.155,258.189 125.922,256.067 127.689,254.013 129.457,252.026 131.224,250.106 132.992,248.252 134.759,246.462 136.527,244.737 
 138.294,243.074 140.062,241.474 141.829,239.936 143.596,238.458 145.364,237.04 147.131,235.681 148.899,234.381 150.666,233.138 152.434,231.952 154.201,230.822 
 155.969,229.747 157.736,228.727 159.503,227.76 161.271,226.846 163.038,225.984 164.806,225.174 166.573,224.414 168.341,223.704 170.108,223.044 171.876,222.431 
 173.643,221.866 175.411,221.349 177.178,220.877 178.945,220.451 180.713,220.07 182.48,219.732 184.248,219.438 186.015,219.187 187.783,218.978 189.55,218.81 
 191.318,218.682 193.085,218.594 194.852,218.546 196.62,218.536 198.387,218.564 200.155,218.629 201.922,218.73 203.69,218.868 205.457,219.04 207.225,219.248 
 208.992,219.489 210.759,219.763 212.527,220.07 214.294,220.409 216.062,220.779 217.829,221.18 219.597,221.611 221.364,222.072 223.132,222.561 224.899,223.079 
 226.666,223.625 228.434,224.197 230.201,224.797 231.969,225.422 233.736,226.072 235.504,226.747 237.271,227.446 239.039,228.169 240.806,228.915 242.574,229.684 
 244.341,230.474 246.108,231.286 247.876,232.118 249.643,232.971 251.411,233.844 253.178,234.735 254.946,235.646 256.713,236.574 258.481,237.521 260.248,238.484 
 262.015,239.464 263.783,240.46 265.55,241.471 267.318,242.498 269.085,243.54 270.853,244.595 272.62,245.664 274.388,246.746 276.155,247.841 277.922,248.948 
 279.69,250.067 281.457,251.197 283.225,252.338 284.992,253.489 286.76,254.65 288.527,255.82 290.295,257 292.062,258.188 293.829,259.384 295.597,260.588 
 297.364,261.799 299.132,263.017 300.899,264.242 302.667,265.473 304.434,266.709 306.202,267.95 307.969,269.197 309.737,270.447 311.504,271.702 313.271,272.961 
 315.039,274.222 316.806,275.487 318.574,276.755 320.341,278.024 322.109,279.295 323.876,280.568 325.644,281.842 327.411,283.117 329.178,284.392 330.946,285.667 
 332.713,286.942 334.481,288.216 336.248,289.49 338.016,290.762 339.783,292.033 341.551,293.302 343.318,294.568 345.085,295.833 346.853,297.094 348.62,298.352 
 350.388,299.608 352.155,300.859 353.923,302.107 355.69,303.35 357.458,304.589 359.225,305.823 360.992,307.052 362.76,308.276 364.527,309.494 366.295,310.707 
 368.062,311.914 369.83,313.114 371.597,314.308 373.365,315.495 375.132,316.676 376.9,317.849 378.667,319.014 380.434,320.173 382.202,321.323 383.969,322.465 
 385.737,323.599 387.504,324.725 389.272,325.842 391.039,326.95 392.807,328.05 394.574,329.14 396.341,330.221 398.109,331.292 399.876,332.354 401.644,333.406 
 403.411,334.448 405.179,335.479 406.946,336.501 408.714,337.512 410.481,338.512 412.248,339.502 414.016,340.481 415.783,341.449 417.551,342.406 419.318,343.352 
 421.086,344.286 422.853,345.209 424.621,346.121 426.388,347.021 428.155,347.909 429.923,348.785 431.69,349.65 433.458,350.502 435.225,351.343 436.993,352.171 
 438.76,352.987 440.528,353.791 442.295,354.583 444.063,355.362 445.83,356.129 447.597,356.884 449.365,357.626 451.132,358.355 452.9,359.072 454.667,359.776 
 456.435,360.468 458.202,361.147 459.97,361.813 461.737,362.467 463.504,363.108 465.272,363.736 467.039,364.352 468.807,364.955 470.574,365.545 472.342,3

 60.162399 seconds (189.52 M allocations: 17.745 GB, 33.18% gc time)


In [51]:
hz(w) = GynC.Hz(w, ys, zs, rho_std)
ll(w) = GynC.logLw(w, ys, datas, rho_std)
gr()
plot(hz.(ws), yaxis="H_z") |> display
plot(ll.(ws), yaxis="logL") |> display
plot((1-reg)*(ll.(ws)) + reg*hz.(ws), yaxis="obj (reg=$reg)") |> display
;

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 7.10 
 
 
 7.15 
 
 
 7.20 
 
 
 H 
 
 
 z 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 58.7073,381.015 69.6738,358.054 80.6402,336.939 91.6067,317.362 102.573,299.102 113.54,282.029 124.506,265.997 135.472,250.904 146.439,236.667 157.405,223.227 
 168.372,210.523 179.338,198.506 190.305,187.136 201.271,176.479 212.238,166.48 223.204,157.066 234.17,148.169 245.137,139.737 256.103,131.742 267.07,124.145 
 278.036,116.92 289.003,110.038 299.969,103.474 310.936,97.2141 321.902,91.2421 332.868,85.5416 343.835,80.0965 354.801,74.8911 365.768,69.9167 376.734,65.1644 
 387.701,60.62 398.667,56.2715 409.633,52.1084 420.6,48.1214 431.566,44.3017 442.533,40.6402 453.499,37.1312 464.466,33.7724 475.432,30.5568 486.399,27.4794 
 497.365,24.5321 508.331,21.7072 519.298,19.0265 530.264,16.5109 541.231,14.1404 552.197,11.9034 563.164,9.77186 574.13,7.73779 585.097,5.79853 596.063,3.93701 
 
 "/>
 
 
 
 
 y1

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -1420 
 
 
 -1410 
 
 
 -1400 
 
 
 logL 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 67.5962,3.93701 78.3813,14.7578 89.1663,24.7088 99.9513,34.1633 110.736,43.3506 121.521,52.3438 132.306,61.2487 143.091,70.1162 153.877,78.9792 164.662,87.8467 
 175.447,96.7248 186.232,105.611 197.017,114.495 207.802,123.31 218.587,132.035 229.372,140.674 240.157,149.233 250.942,157.714 261.727,166.109 272.512,174.428 
 283.297,182.677 294.082,190.86 304.867,198.98 315.652,207.032 326.437,215.009 337.222,222.913 348.007,230.746 358.792,238.506 369.577,246.193 380.362,253.801 
 391.147,261.332 401.932,268.784 412.717,276.156 423.502,283.442 434.287,290.639 445.072,297.752 455.858,304.779 466.643,311.724 477.428,318.59 488.213,325.384 
 498.998,332.1 509.783,338.72 520.568,345.095 531.353,351.1 542.138,356.756 552.923,362.071 563.708,367.144 574.493,371.953 585.278,376.557 596.063,381.015 
 
 "/>
 
 
 
 
 y1

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 7.10 
 
 
 7.15 
 
 
 7.20 
 
 
 obj (reg=1) 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 58.7073,381.015 69.6738,358.054 80.6402,336.939 91.6067,317.362 102.573,299.102 113.54,282.029 124.506,265.997 135.472,250.904 146.439,236.667 157.405,223.227 
 168.372,210.523 179.338,198.506 190.305,187.136 201.271,176.479 212.238,166.48 223.204,157.066 234.17,148.169 245.137,139.737 256.103,131.742 267.07,124.145 
 278.036,116.92 289.003,110.038 299.969,103.474 310.936,97.2141 321.902,91.2421 332.868,85.5416 343.835,80.0965 354.801,74.8911 365.768,69.9167 376.734,65.1644 
 387.701,60.62 398.667,56.2715 409.633,52.1084 420.6,48.1214 431.566,44.3017 442.533,40.6402 453.499,37.1312 464.466,33.7724 475.432,30.5568 486.399,27.4794 
 497.365,24.5321 508.331,21.7072 519.298,19.0265 530.264,16.5109 541.231,14.1404 552.197,11.9034 563.164,9.77186 574.13,7.73779 585.097,5.79853 596.063,3.93701 
 
 "/>
 
 
 
 
 y1